In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
reload(fd)
reload(sd)

print('done')

# Loading raw transactions file

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx.head(5)


# Pie diagram showing the imbalanced data

In [ ]:
import matplotlib.pyplot as plt

print("Class as pie chart fraud are very unbalanced data:")
plt.pie(dfTrx['Class'].value_counts(),autopct='%1.3f%%', labels=['Genuine','Fraud'], colors=['yellowgreen','r'])
if(saveImg):
    plt.savefig('../imgs/FD01B-piediag-transaction.png')
plt.show()


# Count plot showing the imbalanced data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
  
# count plot on single categorical variable
sns.countplot(x ='Class', data = dfTrx)
plt.title("Fraud on credit card transactions")
plt.xlabel("0=Genuine    1=Fraudulent")
plt.ylabel("Nb transactions")
 
# Show the plot
if(saveImg):
    plt.savefig('../imgs/FD01B-count-plot-transaction.png')
plt.show()

In [ ]:
dfTrx.info()

In [ ]:
nan_count = dfTrx.isnull().sum()
print('Number of NaN values:', nan_count)

# issue with cluster profile

In [ ]:
# issue cluster profile
print("All transactions")
print( dfTrx.cluster_profile.value_counts())
print("-----------------------")
print("Fraudulent transactions")
print( dfTrx[dfTrx['Class']==1].cluster_profile.value_counts())
# bug almost all fraudulent transaction have cluster profile equals to UNKNOWN